In [40]:
# CSV Data Ingestion

import pandas as pd
import glob
import shutil
import os
from dotenv import load_dotenv
load_dotenv(override=True)
import pyodbc 
import numpy as np

Concat all CSVs in data file to one df

In [26]:
path = 'Data'
csv_files = glob.glob(path+"/*.csv")
df_raw_data = pd.concat(map(pd.read_csv, csv_files))

Move files to archive

In [27]:
source_dir = 'Data'
target_dir = 'Data/Old'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    shutil.move(os.path.join(source_dir, file_name), target_dir)

Clean column names for whitespace

In [34]:
df_raw_data.columns = [c.replace(' ', '') for c in df_raw_data.columns]

In [43]:
df_raw_data_clean = df_raw_data.replace(np.nan, '', regex=True)
df_raw_data_clean

,BSBNumber,AccountNumber,TransactionDate,Narration,Cheque,Debit,Credit,Balance,TransactionType
0,,5229 8008 3667 1385,25/07/2023,AUTHORISATION ONLY - Enviro Friendly CarwasM ...,,10.3,,-777.89,
1,,5229 8008 3667 1385,25/07/2023,AUTHORISATION ONLY - BLCK BRWN M ...,,45.0,,-767.59,
2,,5229 8008 3667 1385,25/07/2023,AUTHORISATION ONLY - AMAZON AU RETAIL S ...,,6.25,,-722.59,
3,,5229 8008 3667 1385,25/07/2023,AUTHORISATION ONLY - WOOLWORTHS 3149 R ...,,7.0,,-716.34,
4,,5229 8008 3667 1385,25/07/2023,AUTHORISATION ONLY - APPLE.COM/BILL S ...,,14.99,,-709.34,
...,...,...,...,...,...,...,...,...,...
223,,5229 8008 3667 1385,27/04/2023,WOOLWORTHS 3149 RINGWOOD AU,,16.0,,-873.19,WDL
224,,5229 8008 3667 1385,26/04/2023,WOOLWORTHS 3149 RINGWOOD AU,,49.65,,-857.19,WDL
225,,5229 8008 3667 1385,26/04/2023,TREK BICYCLE CORPORATI RINGWOOD AU,,29.95,,-807.54,WDL
226,,5229 8008 3667 1385,26/04/2023,DUSK AUSTRALASIA PTY RINGWOOD,,29.99,,-777.59,WDL


Move files to DB

In [23]:

server = os.getenv('SQL_SERVER')
database = os.getenv('SQL_DATABASE')
cnxn_str = 'DRIVER={SQL Server};server='+server+';Database='+database+';Trusted_Connection=yes;'
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()


In [45]:
for index, row in df_raw_data_clean.iterrows():
    cursor.execute("Insert into raw_txns (BSBNumber,AccountNumber,TransactionDate,Narration,Cheque,Debit,Credit,Balance,TransactionType) Values (?,?,?,?,?,?,?,?,?)", row.BSBNumber,row.AccountNumber,row.TransactionDate,row.Narration,row.Cheque,row.Debit,row.Credit,row.Balance,row.TransactionType)
cnxn.commit()